In [1]:
from supabase.client import Client
from typing import Dict, Any, Tuple, List
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults


In [2]:
from utils.utils import initialize_environment_variables, initialize_subabase_client, initialize_openai_client
from utils.utils import initialize_tavily_client
from api.config import interface_config

initialize_environment_variables("../.env")

# Initialize Supabase Client
supabase_client = initialize_subabase_client()

# Initialize OpenAI Client
openai_client = initialize_openai_client()

# Initialize Tavily Client
tavily_client = initialize_tavily_client()

In [3]:
input_dict = {
"school_type": "Gymnasium",
"subject": "English",
"topic": "Shakespeare",
"grade": "13.",
"state": "Bayern",
"keywords": ["shakespeare", "poetry", "analysis"], 
"context": ["2147550a-800f-4fd1-9021-e5f443c97ce5", "98e1f41e-5985-418b-a067-aad97091ac23", "26392f41-80ab-4c01-a408-4faf36f06291"]
}

input_dict

In [4]:
def unpack_prompt_input(input: Dict[str, Any]) -> Tuple[str, str, str, str, List[str], List[str]]:
    school_type = str(input.get("subject"))
    subject = str(input.get("subject"))
    topic = str(input.get("topic"))
    grade = str(input.get("grade"))
    state = str(input.get("state"))
    keywords = [str(i) for i in input.get("keywords")]
    context = [str(i) for i in input.get("context")]
    return school_type, subject, topic, grade, state, keywords, context

In [14]:
prompt = """Plane eine Unterrichtsstunde im Fach {subject} zum Thema {topic} für eine {grade} Klasse in {state} für den Schultyp {school_type}.

Schritt 1: Finde aktuelle Informationen zu den Schlüsselbegriffen!
Schritt 2: (Optional) Suche mehr Kontext
Schritt 3: Formatiere deine Antwort in der folgenden json-Struktur:
______________________________

{json_format}
______________________________

Los geht's!
"""
prompt = PromptTemplate.from_template(prompt)

In [15]:
prompt

PromptTemplate(input_variables=['grade', 'json_format', 'school_type', 'state', 'subject', 'topic'], template="Plane eine Unterrichtsstunde im Fach {subject} zum Thema {topic} für eine {grade} Klasse in {state} für den Schultyp {school_type}.\n\nSchritt 1: Finde aktuelle Informationen zu den Schlüsselbegriffen!\nSchritt 2: (Optional) Suche mehr Kontext\nSchritt 3: Formatiere deine Antwort in der folgenden json-Struktur:\n______________________________\n\n{json_format}\n______________________________\n\nLos geht's!\n")

In [8]:
school_type, subject, topic, grade, state, keywords, context = unpack_prompt_input(input_dict)

In [18]:
json_format = """{{
    "learn_goals": "xyz",
    "table_data": [
        {{"title": "abc", "duration": "10min", "content": "blabla"}},
        {{...}},
        {{...}}
    ]
}}"""

def create_task_template(prompt: PromptTemplate, school_type, subject, topic, grade, state, json_format):
    prompt.format({"subject": subject, "topic": topic})
    prompt.format(topic=topic)
    prompt.format(grade=grade)
    prompt.format(state=state)
    prompt.format(school_type=school_type)
    prompt.format(json_format)

task = create_task_template(prompt, school_type, subject, topic, grade, state, json_format)
print(task)

KeyError: 'topic'

def get_uploaded_ids(filepath, upload_table_name):
    data, _cnt = supabase_client.table(upload_table_name).select('id').contains('metadata', {'source':filepath}).execute()
    return [data[1][i].get("id") for i in range(len(data[1]))]

def create_filter_list(context):
    uploaded_ids = []
    for doc in context:
        if len(doc) > 0:
            doc_uploaded_ids = get_uploaded_ids(doc, interface_config.upload_table_name)
            if len(doc_uploaded_ids) > 0:
                uploaded_ids += doc_uploaded_ids
    return uploaded_ids

In [ ]:
filter_list = context

In [ ]:
def initialize_vector_store(client: Client, embedding: OpenAIEmbeddings, table_name: str, query_name: str):
    vector_store = SupabaseVectorStore(
        client=client,
        embedding=embedding,
        table_name=table_name,
        query_name=query_name,
    )
    return vector_store

In [ ]:
embedding = OpenAIEmbeddings()

In [ ]:
vector_store = initialize_vector_store(supabase_client, embedding, interface_config.upload_table_name, interface_config.supabase_match_function) 

In [ ]:
retrieved_context = vector_store.similarity_search(task,k=interface_config.context_k,filter=filter_list)

2024-02-03 03:22:59,364:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-03 03:22:59,533:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_with_uuid_filter?limit=1 "HTTP/1.1 200 OK"


In [ ]:
print(type(retrieved_context[0]))

<class 'langchain_core.documents.base.Document'>


In [ ]:
retrieved_context[0]

Document(page_content='ENTWURF RICHTLINIEN 02.08.2023    \n 15 , Erfahrungs- und Lebensraum. Eine kind- und jugendgerechte Ganztagsbildung er-\nmöglicht die Einbindung zusätzlicher Lernangebote und orientiert sich an den Bedürf-\nnissen der Kinder und Jugendlichen. Ganztagsangebote werden mit vielfältigen Part-\nnern umgesetzt. Zentrale Merkmale des Ganztags sind u.a. Konzepte zur Förderung \nbesonderer Interessen und Bedarfe der Schülerinnen und Schüler, zusätzliche Zu-\ngänge zum Lernen, Angebote z.B. aus den Bereichen Sport, Kultur, Berufsorientierung \nund Gesundheit sowie Freiräume für Selbstbildungsprozesse und selbstbestimmte Ak-\ntivitäten.  \nZu den Zielen ganztägiger Bildung trägt ebenso die Organisationsform fakultativer Ar-\nbeitsgemeinschaften bei. Diese sind in Ausrichtung und Zielsetzung nicht gebunden \nan curriculare Vorgaben und konstituieren sich aus den Neigungen, Interessen und \nBedarfen von Schülerinnen und Schülern sowie aus Impulsen der Lehrkräfte und wei-\nter

In [ ]:
def create_prompt_template(retrieved_context, keywords) -> str:
    '\n'.join(keywords)
    return f"""Du bist ein Assistent für Lehrkräfte und deine Aufgabe ist Unterricht strukturiert, fachlich korrekt, und detailliert vorzubereiten. 
Schlüsselbegriffe helfen dir bei der Lösung der Aufgabe die richtigen Themen zu wählen.

Schlüsselbegriffe:
------------------
{keywords}
------------------

Die Lehrkraft stellt dir Kontext zur Lösung der Aufgabe in deiner Bibliothek zur Verfügung. 
Hier ist ein kurzer Ausschnitt aus dem Kontext in deiner Bibliothek:

Ausschnitt aus dem Kontext:
---------
{retrieved_context}
---------

Die Antwort soll in jedem Fall eine inhaltliche und zeitliche Struktur für den Unterricht beinhalten.
Die Gesamtlänge des Unterrichts soll insgesamt 45 Minuten betragen.
Die Antwort darf keine Wiederholungen enthalten!
"""
prompt_template = create_prompt_template(retrieved_context, keywords)

In [ ]:
print(prompt_template)

Du bist ein Assistent für Lehrkräfte und deine Aufgabe ist Unterricht strukturiert, fachlich korrekt, und detailliert vorzubereiten. 
Schlüsselbegriffe helfen dir bei der Lösung der Aufgabe die richtigen Themen zu wählen.

Schlüsselbegriffe:
------------------
['shakespeare', 'poetry', 'analysis']
------------------

Die Lehrkraft stellt dir Kontext zur Lösung der Aufgabe in deiner Bibliothek zur Verfügung. 
Hier ist ein kurzer Ausschnitt aus dem Kontext in deiner Bibliothek:

Ausschnitt aus dem Kontext:
---------
[Document(page_content='ENTWURF RICHTLINIEN 02.08.2023    \n 15 , Erfahrungs- und Lebensraum. Eine kind- und jugendgerechte Ganztagsbildung er-\nmöglicht die Einbindung zusätzlicher Lernangebote und orientiert sich an den Bedürf-\nnissen der Kinder und Jugendlichen. Ganztagsangebote werden mit vielfältigen Part-\nnern umgesetzt. Zentrale Merkmale des Ganztags sind u.a. Konzepte zur Förderung \nbesonderer Interessen und Bedarfe der Schülerinnen und Schüler, zusätzliche Zu-\ngä

In [ ]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
prompt.messages[0].prompt.template = prompt_template

In [ ]:
prompt.messages[0].dict()['prompt']['template']

"Du bist ein Assistent für Lehrkräfte und deine Aufgabe ist Unterricht strukturiert, fachlich korrekt, und detailliert vorzubereiten. \nSchlüsselbegriffe helfen dir bei der Lösung der Aufgabe die richtigen Themen zu wählen.\n\nSchlüsselbegriffe:\n------------------\n['shakespeare', 'poetry', 'analysis']\n------------------\n\nDie Lehrkraft stellt dir Kontext zur Lösung der Aufgabe in deiner Bibliothek zur Verfügung. \nHier ist ein kurzer Ausschnitt aus dem Kontext in deiner Bibliothek:\n\nAusschnitt aus dem Kontext:\n---------\n[Document(page_content='ENTWURF RICHTLINIEN 02.08.2023    \\n 15 , Erfahrungs- und Lebensraum. Eine kind- und jugendgerechte Ganztagsbildung er-\\nmöglicht die Einbindung zusätzlicher Lernangebote und orientiert sich an den Bedürf-\\nnissen der Kinder und Jugendlichen. Ganztagsangebote werden mit vielfältigen Part-\\nnern umgesetzt. Zentrale Merkmale des Ganztags sind u.a. Konzepte zur Förderung \\nbesonderer Interessen und Bedarfe der Schülerinnen und Schüler, 

In [ ]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=interface_config.tavily_max_response)

In [ ]:
embeddings = OpenAIEmbeddings()
vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="vector_store",
        query_name="match_documents_with_uuid_filter",
    )
retriever = vector_store.as_retriever(search_type=interface_config.retriever_search_type, search_kwargs={"k": interface_config.retriever_k, 'lambda_mult': interface_config.retriever_mult, 'filter': filter_list})
retriever_tool = tool = create_retriever_tool(
    retriever,
    "get_more_context",
    "A personal library optimized for user defined context information. Useful for when you need to know more about the given context. Input should be a search query.",
)

In [ ]:
tools = [retriever_tool, tavily_tool]

In [ ]:
llm = ChatOpenAI(model_name=interface_config.gpt_model_version, temperature=interface_config.gpt_model_temperature, response_format={ "type": "json_object" })
agent = create_openai_functions_agent(llm, tools, prompt)

/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:299: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke({"input": task})



> Entering new AgentExecutor chain...


KeyError: "'page'"

In [ ]:
print(type(retrieved_context[0]))

<class 'langchain_core.documents.base.Document'>


In [ ]:
context_dict = [doc.dict() for doc in retrieved_context]

In [ ]:
len(context_dict)

1

In [ ]:
response["context"] = context_dict

In [ ]:
type(response)

dict

In [ ]:
response

{'input': 'Du bist ein Assistent für Lehrkräfte und deine Aufgabe ist Unterricht strukturiert, fachlich korrekt, und detailliert vorzubereiten. \nSchlüsselbegriffe helfen dir bei der Lösung der Aufgabe die richtigen Themen zu wählen.\n\nSchlüsselbegriffe:\n------------------\n[\'shakespeare\', \'poetry\', \'analysis\']\n------------------\n\nDie Lehrkraft stellt dir Kontext zur Lösung der Aufgabe in deiner Bibliothek zur Verfügung. \nHier ist ein kurzer Ausschnitt aus dem Kontext in deiner Bibliothek:\n\nAusschnitt aus dem Kontext:\n---------\n[Document(page_content=\'ENTWURF RICHTLINIEN 02.08.2023    \\n 15 , Erfahrungs- und Lebensraum. Eine kind- und jugendgerechte Ganztagsbildung er-\\nmöglicht die Einbindung zusätzlicher Lernangebote und orientiert sich an den Bedürf-\\nnissen der Kinder und Jugendlichen. Ganztagsangebote werden mit vielfältigen Part-\\nnern umgesetzt. Zentrale Merkmale des Ganztags sind u.a. Konzepte zur Förderung \\nbesonderer Interessen und Bedarfe der Schülerin

In [ ]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser


In [ ]:
for (key, value) in response.items():
    print(f"key: {type(key)}")
    print(f"value: {type(value)}")

key: <class 'str'>
value: <class 'str'>
key: <class 'str'>
value: <class 'str'>
key: <class 'str'>
value: <class 'list'>


In [ ]:
print(type(response))

<class 'dict'>


In [ ]:
import json
output = json.dumps(response)

In [ ]:
output

'{"input": "Du bist ein Assistent f\\u00fcr Lehrkr\\u00e4fte und deine Aufgabe ist Unterricht strukturiert, fachlich korrekt, und detailliert vorzubereiten. \\nSchl\\u00fcsselbegriffe helfen dir bei der L\\u00f6sung der Aufgabe die richtigen Themen zu w\\u00e4hlen.\\n\\nSchl\\u00fcsselbegriffe:\\n------------------\\n[\'shakespeare\', \'poetry\', \'analysis\']\\n------------------\\n\\nDie Lehrkraft stellt dir Kontext zur L\\u00f6sung der Aufgabe in deiner Bibliothek zur Verf\\u00fcgung. \\nHier ist ein kurzer Ausschnitt aus dem Kontext in deiner Bibliothek:\\n\\nAusschnitt aus dem Kontext:\\n---------\\n[Document(page_content=\'ENTWURF RICHTLINIEN 02.08.2023    \\\\n 15 , Erfahrungs- und Lebensraum. Eine kind- und jugendgerechte Ganztagsbildung er-\\\\nm\\u00f6glicht die Einbindung zus\\u00e4tzlicher Lernangebote und orientiert sich an den Bed\\u00fcrf-\\\\nnissen der Kinder und Jugendlichen. Ganztagsangebote werden mit vielf\\u00e4ltigen Part-\\\\nnern umgesetzt. Zentrale Merkmale de

In [ ]:
def download_file_from_bucket(destination: str, full_path: str, path: str):
    full_path_list = full_path.split("/")
    bucket_name = full_path_list[0]
    with open(destination, 'wb+') as f:
        storage_file = supabase_client.storage.from_(bucket_name).download(path)
        f.write(storage_file)

In [ ]:
download_file_from_bucket("RMH-Rechnung_12_23.pdf", "context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf", "user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf")

2024-02-03 03:09:16,354:INFO - HTTP Request: GET https://smxwnqdpxcvclxhbvhsf.supabase.co/storage/v1/object/context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf "HTTP/1.1 200 OK"


In [ ]:
upload_dict = {"path":"user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf","id":"c2f81078-039d-435a-b0c6-84534ac9048f","fullPath":"context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf"}

In [ ]:
full_path = upload_dict['fullPath']

In [ ]:
full_path

'context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf'

In [ ]:
full_path.split("/")

['context_uploads',
 'user_2e4ace06-270b-429b-9726-f937261ed293',
 'RMH-Rechnung_12_23.pdf']

In [ ]:
import os
import tempfile
import mimetypes
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename

from api.files_service import allowed_file, download_file_from_bucket, upload_text, upload_pdf, get_uploaded_ids
from api.ai_service import getanswer


file_path = secure_filename("user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf")  
print(file_path)  
download_file_from_bucket(file_path, "context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf", "user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf")
print(mimetypes.guess_type(file_path))
if mimetypes.guess_type(file_path)[0] == 'text/plain':
    upload_text(file_path)
elif mimetypes.guess_type(file_path)[0] == 'application/pdf':
    print(file_path)
    upload_pdf(file_path)
vector_store_ids = get_uploaded_ids(file_path, interface_config.upload_table_name)


2024-02-03 03:09:16,777:INFO - HTTP Request: GET https://smxwnqdpxcvclxhbvhsf.supabase.co/storage/v1/object/context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf "HTTP/1.1 200 OK"


user_2e4ace06-270b-429b-9726-f937261ed293_RMH-Rechnung_12_23.pdf
('application/pdf', None)
user_2e4ace06-270b-429b-9726-f937261ed293_RMH-Rechnung_12_23.pdf


2024-02-03 03:09:17,506:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-03 03:09:17,738:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/vector_store "HTTP/1.1 201 Created"
2024-02-03 03:09:17,801:INFO - HTTP Request: GET https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/vector_store?select=id&metadata=cs.%7B%22source%22%3A%20%22user_2e4ace06-270b-429b-9726-f937261ed293_RMH-Rechnung_12_23.pdf%22%7D "HTTP/1.1 200 OK"


In [ ]:
{
  "id": "chatcmpl-123",
  "object": "chat.completion",
  "created": 1677652288,
  "model": "gpt-3.5-turbo-0613",
  "system_fingerprint": "fp_44709d6fcb",
  "choices": [{
    "index": 0,
    "message": {
      "role": "assistant",
      "content": "\n\nHello there, how may I assist you today?",
    },
    "logprobs": null,
    "finish_reason": "stop"
  }],
  "usage": {
    "prompt_tokens": 9,
    "completion_tokens": 12,
    "total_tokens": 21
  }
}


NameError: name 'null' is not defined